# Step 8: Model Deployment Using FastAPI

### Objectives:
- Load the best model from W&B.
- Create a FastAPI-based RESTful API for serving predictions.
- Accept input as JSON and return predictions in JSON format.
- Deploy and test the API locally.

### 1. Install & Import Required Libraries

In [1]:
# Install FastAPI and Uvicorn
!pip install fastapi uvicorn wandb joblib


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import wandb
import joblib
import json
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd

### 2. Load the Best Model from W&B

In [10]:
import wandb
import joblib

# Initialize W&B to load the best model
wandb.init(project="Bank_Marketing_MLOps", name="load-best-model")

# Correct way to load artifact
artifact = wandb.use_artifact("best_model:latest", type="model")  # Use 'latest' alias
artifact_dir = artifact.download()

# Load the model
model_path = f"{artifact_dir}/best_model.pkl"
model = joblib.load(model_path)

print(f"✅ Model loaded successfully from: {model_path}")

wandb:   1 of 1 files downloaded.  


✅ Model loaded successfully from: C:\Users\PC\Documents\MLOps\Bank_Marketing_Project\artifacts\best_model-v0/best_model.pkl


### 3. Create a FastAPI Application

In [15]:
# Create FastAPI app
app = FastAPI(title="Bank Marketing Prediction API", version="1.0")

# Define request input schema with all required features
class InputData(BaseModel):
    age: int
    job: str
    marital: str
    education: str
    default: str
    balance: float
    housing: str
    loan: str
    contact: str
    day: int
    month: str
    duration: int
    campaign: int
    pdays: int
    previous: int
    poutcome: str
    emp_var_rate: float
    cons_price_idx: float
    cons_conf_idx: float
    euribor3m: float
    nr_employed: float
    day_of_week: str

# Define prediction endpoint
@app.post("/predict")
def predict(data: InputData):
    try:
        # Convert input JSON to dictionary
        input_dict = data.dict()

        # Rename fields to match dataset column names
        corrected_input = {
            "age": input_dict["age"],
            "job": input_dict["job"],
            "marital": input_dict["marital"],
            "education": input_dict["education"],
            "default": input_dict["default"],
            "balance": input_dict["balance"],
            "housing": input_dict["housing"],
            "loan": input_dict["loan"],
            "contact": input_dict["contact"],
            "day": input_dict["day"],
            "month": input_dict["month"],
            "duration": input_dict["duration"],
            "campaign": input_dict["campaign"],
            "pdays": input_dict["pdays"],
            "previous": input_dict["previous"],
            "poutcome": input_dict["poutcome"],
            "emp.var.rate": input_dict["emp_var_rate"],   # Convert name
            "cons.price.idx": input_dict["cons_price_idx"], # Convert name
            "cons.conf.idx": input_dict["cons_conf_idx"], # Convert name
            "euribor3m": input_dict["euribor3m"],
            "nr.employed": input_dict["nr_employed"],
            "day_of_week": input_dict["day_of_week"]
        }

        # Convert corrected input to DataFrame
        input_df = pd.DataFrame([corrected_input])

        # Make prediction with probability scores
        probas = model.predict_proba(input_df)[0]  # Get probability of "no" and "yes"
        prediction = model.predict(input_df)[0]
        predicted_label = "yes" if prediction == 1 else "no"

        return {
            "prediction": predicted_label,
            "probability_no": round(probas[0], 4),  # Probability for "no"
            "probability_yes": round(probas[1], 4)  # Probability for "yes"
        }

    except Exception as e:
        return {"error": str(e)}

### 4. Running the FastAPI Server Locally

In [ ]:
import nest_asyncio

# Apply nest_asyncio to allow running FastAPI inside Jupyter
nest_asyncio.apply()

# Run FastAPI within Jupyter Notebook
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [16488]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
C:\Users\PC\AppData\Local\Temp\ipykernel_16488\3076177647.py:34: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  input_dict = data.dict()


INFO:     127.0.0.1:61494 - "POST /predict HTTP/1.1" 200 OK


C:\Users\PC\AppData\Local\Temp\ipykernel_16488\3076177647.py:34: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  input_dict = data.dict()


INFO:     127.0.0.1:61517 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:61525 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:61528 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:61528 - "GET /openapi.json HTTP/1.1" 200 OK
